In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

### Load the data

In [2]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Undersampling by ethnicity

In [3]:
e = data["Ethnicity"]
print('Original dataset shape %s' % Counter(e))

ethnicity_counts = dict(data["Ethnicity"].value_counts())
num_ethnicities = len(ethnicity_counts)
min_count = min(ethnicity_counts.values())

strategy_under = {ethnicity: min_count for ethnicity in range(num_ethnicities)}
under = RandomUnderSampler(sampling_strategy=strategy_under)
print(strategy_under, sep='\n')

data_under, e_under=under.fit_resample(data, e)
print('Undersampled dataset shape %s' % Counter(e_under))

Original dataset shape Counter({0: 1278, 1: 454, 3: 211, 2: 206})
{0: 206, 1: 206, 2: 206, 3: 206}
Undersampled dataset shape Counter({0: 206, 1: 206, 2: 206, 3: 206})


### Split the data

In [4]:
X, y = data_under.drop("Diagnosis", axis=1), data_under["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
1922,6673,80,0,3,2,21.826513,0,14.859331,2.649062,3.551083,...,7.218262,4.826436,1,0,1.266847,1,0,0,0,0
1707,6458,71,0,0,2,17.545005,1,3.782624,6.170481,5.586319,...,20.042813,8.317205,1,1,3.861284,0,1,0,1,1
1022,5773,90,0,2,2,22.189842,0,14.640251,6.858580,1.019058,...,8.726399,9.486590,0,0,4.964514,1,0,0,0,0
1915,6666,80,0,1,1,28.403281,0,11.736276,2.199039,7.889884,...,7.305482,0.630607,0,0,7.154929,0,0,0,0,0
912,5663,71,0,2,1,27.981424,0,10.235515,5.604944,2.004545,...,6.847750,9.734456,0,0,3.687826,0,0,0,0,0


### Train the model

In [5]:
classifier = HistGradientBoostingClassifier(random_state=12)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       134
           1       0.89      0.88      0.88        72

    accuracy                           0.92       206
   macro avg       0.91      0.91      0.91       206
weighted avg       0.92      0.92      0.92       206



In [7]:
X["Ethnicity"].value_counts()

Ethnicity
0    206
1    206
2    206
3    206
Name: count, dtype: int64

### Compare performance based on ethnicity

In [8]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [9]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        34
           1       0.93      0.93      0.93        14

    accuracy                           0.96        48
   macro avg       0.95      0.95      0.95        48
weighted avg       0.96      0.96      0.96        48



In [10]:
# Ethncity 1
eth1_results = results[results["Ethnicity"] == 1]
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.90      0.85      0.88        33
           1       0.69      0.79      0.73        14

    accuracy                           0.83        47
   macro avg       0.80      0.82      0.80        47
weighted avg       0.84      0.83      0.83        47



In [11]:
# Ethnicity 2  
eth2_results = results[results["Ethnicity"] == 2]
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94        31
           1       0.96      0.88      0.92        25

    accuracy                           0.93        56
   macro avg       0.93      0.92      0.93        56
weighted avg       0.93      0.93      0.93        56



In [12]:
# Ethncity 3
eth3_results = results[results["Ethnicity"] == 3]
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96        36
           1       0.94      0.89      0.92        19

    accuracy                           0.95        55
   macro avg       0.95      0.93      0.94        55
weighted avg       0.95      0.95      0.95        55



In [13]:
%load_ext autoreload
%autoreload 2
from fairness_metrics import demographic_parity, equalized_odds, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

for i in [1,2,3]:
    dp = round(demographic_parity(results, 0, i), 4)
    eo = equalized_odds(results, 0, i)
    tpr_diff, fpr_diff = round(eo[0], 4), round(eo[1], 4)
    di = round(disparate_impact(results, 0, i), 4)
    print(dp, tpr_diff, fpr_diff, di)

0.0488 -0.1429 0.1221 0.8568
0.119 -0.0486 0.0028 0.7101
0.0356 -0.0338 -0.0016 0.8912
